# 🚀 Proyecto ELT con SpaceX API  
## Notebook 01 – Extracción y Carga

En este notebook se realiza la **extracción de datos desde la API pública de SpaceX** y el **almacenamiento en Delta Lake**.  

Se siguen los pasos de la consigna:  
1. Extracción de **2 o más endpoints**.  
2. Uso de al menos un **endpoint dinámico (actualizable)** y otro **estático**.  
3. Guardado en **formato Delta Lake**.  
4. Aplicación de **extracción incremental y full** según corresponda.  


In [ ]:
# =========================
# CELDA DE CONFIGURACIÓN INICIAL
# =========================

import sys
from pathlib import Path

# --- RUTA DEL PROYECTO ---
# Detecta automáticamente la raíz del proyecto buscando la carpeta "src"
def find_project_root(marker="src"):
    path = Path().cwd()
    for _ in range(5):  # sube hasta 5 niveles si es necesario
        if (path / marker).exists():
            return path
        path = path.parent
    raise FileNotFoundError(f"No se encontró la carpeta '{marker}' en los niveles superiores.")

project_root = find_project_root()
if str(project_root) not in sys.path:
    sys.path.append(str(project_root))

print(f"Project root detectado en: {project_root}")

# --- IMPORTS DEL PROYECTO ---
from src.extract import fetch_data, filter_incremental
from src.load import save_to_parquet
from src.utils import setup_logger
from src.transform import load_from_parquet

logger = setup_logger()

## 🔗 Endpoints seleccionados

- **Dinámicos:**  
  - `launches/latest` → Último lanzamiento.  
  - `launches/upcoming` → Próximos lanzamientos.  

- **Estáticos:**  
  - `rockets` → Información de cohetes.  
  - `dragons` → Información de cápsulas Dragon.  

In [ ]:
# Extracción de datos

# FULL extraction de Rockets
logger.info("Extrayendo Rockets (FULL)...")
rockets_df = fetch_data("rockets")
display(rockets_df.head())

# FULL extraction de Dragons
logger.info("Extrayendo Dragons (FULL)...")
dragons_df = fetch_data("dragons")
display(dragons_df.head())


print(f"✅ Extracción completa: {len(rockets_df)} rockets extraídos.")
print(f"✅ Extracción completa: {len(dragons_df)} dragons extraídos.")

In [ ]:
# Extracción INCREMENTAL de upcoming launches
logger.info("Extrayendo Upcoming Launches (INCREMENTAL)...")
launches_new = fetch_data("upcoming_launches")
display(launches_new.head())

# Cargar histórico desde Bronze
launches_old = load_from_parquet("upcoming_launches", layer="bronze")

# Filtrar solo lo nuevo
launches_inc = filter_incremental(launches_new, launches_old, time_col="date_utc")

if not launches_inc.empty:
    print(f"✅ Extracción completa: {len(launches_inc)} launches nuevos extraídos.")
else:
    logger.info("No hay datos nuevos para upcoming_launches")

## 💾 Guardado

Se guarda cada DataFrame en formato **Delta Lake**:  
- Los **endpoints dinámicos** (`latest_launch`, `upcoming_launches`) se almacenan con **particiones por fecha (extracción incremental)**.  
- Los **endpoints estáticos** (`rockets`, `dragons`) se guardan en una única ruta (extracción full).  

In [ ]:
# Guardado de datos en Parquet

# FULL save de Rockets
save_to_parquet(rockets_df, "rockets", layer="bronze", incremental=False, mode="overwrite")

# INCREMENTAL save de Launches
if not launches_inc.empty:
    save_to_parquet(launches_inc, "upcoming_launches", layer="bronze", incremental=True, mode="append")

logger.info("Todos los datasets fueron guardados correctamente en Parquet.")